In [53]:
import pandas as pd
import wget
from zipfile import ZipFile
import os
from plotnine import *


In [54]:

diretorio_atual = os.getcwd()

os.chdir(f'{diretorio_atual}')

In [55]:
diretorio_atual

'c:\\Users\\myrra\\Documents\\Financas\\Magic Formula\\CVM\\dados_cvm_dfp\\DFP'

In [56]:
anos = range(2013, 2024)

url = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'


In [57]:
dados_dfp = []

for ano in anos:
    dados_dfp.append(f'dfp_cia_aberta_{ano}.zip')


In [58]:
for i in dados_dfp:
    wget.download(url + i)

In [59]:
# Extrai os arquivos ZIP baixados
for i in dados_dfp:
    ZipFile(i, 'r').extractall('Planilhas')

In [60]:
# Lista dos nomes dos arquivos CSV a serem unificados
nomes = ['DRE_con', 'DRE_ind']

# Itera sobre cada nome de arquivo para unificar os dados
for nome in nomes:
    arquivo = pd.DataFrame()
    # Itera sobre cada ano para ler os dados de cada arquivo CSV e unificá-los
    for ano in anos:
        # Caminho do arquivo CSV para o ano e nome especificados
        caminho = 'Planilhas\dfp_cia_aberta_{}_{}.csv'.format(nome, ano)
        # Lê o arquivo CSV e adiciona ao DataFrame 'arquivo'
        arquivo = pd.concat([arquivo, pd.read_csv(caminho, sep=';', decimal=',', encoding='ISO-8859-1')])
    
    # Salva o DataFrame unificado como um novo arquivo CSV
    arquivo.to_csv(f'Dados_unificados/dfp_cia_aberta_{nome}_2013-2023.csv', index=False)

In [82]:
# Leitura doe dados e exclusao das linhas denominadas PENÚLTIMO
dre = pd.read_csv('Dados_unificados/dfp_cia_aberta_DRE_con_2013-2023.csv')
dre = dre[dre['ORDEM_EXERC'] == 'ÚLTIMO']
dre

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.000.000/0001-91,2013-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2013-01-01,2013-12-31,3.01,Receitas da Intermediação Financeira,1.045822e+08,S
3,00.000.000/0001-91,2013-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2013-01-01,2013-12-31,3.01.01,Receita de Juros,1.045822e+08,S
5,00.000.000/0001-91,2013-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2013-01-01,2013-12-31,3.02,Despesas da Intermediação Financeira,-7.437689e+07,S
7,00.000.000/0001-91,2013-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2013-01-01,2013-12-31,3.02.01,Despesas de Juros,-6.284812e+07,S
9,00.000.000/0001-91,2013-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2013-01-01,2013-12-31,3.02.02,Despesa com Provisão para Perdas em Empréstimos,-1.151418e+07,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286516,97.837.181/0001-47,2023-12-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-01-01,2023-12-31,3.99,Lucro por Ação - (Reais / Ação),0.000000e+00,N
286518,97.837.181/0001-47,2023-12-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-01-01,2023-12-31,3.99.01,Lucro Básico por Ação,0.000000e+00,N
286520,97.837.181/0001-47,2023-12-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-01-01,2023-12-31,3.99.01.01,ON,9.772000e-01,N
286522,97.837.181/0001-47,2023-12-31,1,DEXCO S.A.,21091,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2023-01-01,2023-12-31,3.99.02,Lucro Diluído por Ação,0.000000e+00,N


In [107]:
# Filtro das contas, dos codigos CVM e do exercício
dados = dre[dre.CD_CONTA.isin(['3.01', #Receita de bens e/ou serviços 
                               '3.03', # Resultado Bruto
                               '3.05', # EBIT
                               '3.11' # Lucro/Prejuízo Consolidado no Período
                               ]) #& dre.CD_CVM.isin([12190, 26700, 21431, 19550])]
]

# Selecionar Colunas
dados = dados[['DT_REFER', 'DENOM_CIA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA']]

dados

,DT_REFER,DENOM_CIA,CD_CONTA,DS_CONTA,VL_CONTA
1,2013-12-31,BCO BRASIL S.A.,3.01,Receitas da Intermediação Financeira,104582211.0
13,2013-12-31,BCO BRASIL S.A.,3.03,Resultado Bruto Intermediação Financeira,30205323.0
51,2013-12-31,BCO BRASIL S.A.,3.05,Resultado Antes dos Tributos sobre o Lucro,12858994.0
83,2013-12-31,BRB BCO DE BRASILIA S.A.,3.01,Receitas da Intermediação Financeira,1910786.0
87,2013-12-31,BRB BCO DE BRASILIA S.A.,3.03,Resultado Bruto Intermediação Financeira,1366562.0
...,...,...,...,...,...
286442,2023-12-31,LOJAS QUERO QUERO S.A.,3.11,Lucro/Prejuízo Consolidado do Período,19679.0
286458,2023-12-31,DEXCO S.A.,3.01,Receita de Venda de Bens e/ou Serviços,7383409.0
286466,2023-12-31,DEXCO S.A.,3.03,Resultado Bruto,2375867.0
286486,2023-12-31,DEXCO S.A.,3.05,Resultado Antes do Resultado Financeiro e dos ...,1269654.0


In [114]:
# Conversao da coluna data
dados['DT_REFER'] = pd.to_datetime(dados['DT_REFER'])

# Criar a coluna Ano
dados['Ano'] = dados['DT_REFER'].dt.year

# Removendo duplicatas baseadas nas colunas de interesse
dados_sem_duplicatas = dados.drop_duplicates(subset=['DENOM_CIA'])

# Tentativa de pivotear o DataFrame limpo
dados_pivot = dados_sem_duplicatas.pivot(index=['Ano', 'DENOM_CIA'], columns='CD_CONTA', values='VL_CONTA')


In [115]:
dados_pivot.head(20)

CD_CONTA                                                     3.01
Ano  DENOM_CIA                                                   
2013 ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.         0.0
     AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.               700755.0
     AES ELPA S.A.                                      9012207.0
     AES TIETE ENERGIA SA                               9428650.0
     AES TIETE SA                                       2336933.0
     ALGAR TELECOM S/A                                  1901995.0
     ALIANSCE SHOPPING CENTERS S.A.                      448421.0
     ALIANSCE SONAE SHOPPING CENTERS S.A.                275754.0
     ALLIS PARTICIPAÇÕES S.A.                            297889.0
     ALPARGATAS S.A.                                    3425959.0
     ALPER CONSULTORIA E CORRETORA DE SEGUROS S.A.       257848.0
     ALUPAR INVESTIMENTO S/A                            1290831.0
     AMBEV S.A.                                        34791391.0
     AMERICANAS S.A.                                    6088500.0
     ANDRADE GUTIERREZ CONCESSOES S.A.                    41865.0
     ANDRADE GUTIERREZ PARTICIPACOES S.A.                 41865.0
     ANHANGUERA EDUCACIONAL PARTICIPAÇÕES S.A.          1812782.0
     ANIMA HOLDING S.A.                                  461292.0
     ARAUCÁRIA PARTICIPAÇÕES SA                          109191.0
     AREZZO INDÚSTRIA E COMÉRCIO S.A.                    962950.0

In [111]:
## Indicadores  
indicadores = (dados_pivot
               .assign(margem_bruta = (dados_pivot['3.03']) / dados_pivot['3.01'] * 100, 
                       margem_liquida = (dados_pivot['3.11']) / dados_pivot['3.01'] * 100, 
                       margem_ebit = (dados_pivot['3.05']) / dados_pivot['3.01'] * 100))

# Retira o Indice
indicadores.reset_index(inplace=True)

CD_CONTA,Ano,DENOM_CIA,3.01,3.03,3.05,3.11,margem_bruta,margem_liquida,margem_ebit
0,2013,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,0.0,-1603.0,-17819.0,-25410.0,-inf,-inf,-inf
1,2013,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,700755.0,312611.0,184858.0,80572.0,44.610599,11.497884,26.379833
2,2013,AES ELPA S.A.,9012207.0,291492.0,245546.0,150571.0,3.234413,1.670745,2.724593
3,2013,AES TIETE ENERGIA SA,9428650.0,1765863.0,1624370.0,1063465.0,18.728694,11.279080,17.228023
4,2013,AES TIETE SA,2336933.0,1362175.0,1362175.0,881359.0,58.289005,37.714346,58.289005
...,...,...,...,...,...,...,...,...,...
672,2022,TECIDOS E ARMARINHOS MIGUEL BARTOLOMEU S.A.,4468988.0,1102140.0,640959.0,440973.0,24.661959,9.867402,14.342375
673,2022,TRÊS TENTOS AGROINDUSTRIAL S/A,6885839.0,1043299.0,537651.0,571174.0,15.151371,8.294908,7.808068
674,2022,V. TAL - REDE NEUTRA DE TELECOMUNICAÇÕES S.A.,5119719.0,545041.0,265086.0,-443340.0,10.645916,-8.659460,5.177745
675,2022,VOTORANTIM CIMENTOS S.A.,25797366.0,4814826.0,2974861.0,1145454.0,18.664022,4.440197,11.531646


In [116]:
# Arredodamento dos valores
indicadores = indicadores.round(decimals = 3)
indicadores.head()

CD_CONTA,Ano,DENOM_CIA,3.01,3.03,3.05,3.11,margem_bruta,margem_liquida,margem_ebit
0,2013,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,0.0,-1603.0,-17819.0,-25410.0,-inf,-inf,-inf
1,2013,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,700755.0,312611.0,184858.0,80572.0,44.611,11.498,26.380
2,2013,AES ELPA S.A.,9012207.0,291492.0,245546.0,150571.0,3.234,1.671,2.725
3,2013,AES TIETE ENERGIA SA,9428650.0,1765863.0,1624370.0,1063465.0,18.729,11.279,17.228
4,2013,AES TIETE SA,2336933.0,1362175.0,1362175.0,881359.0,58.289,37.714,58.289


In [117]:
# Selecionar colunas e arredondar valores
indicadores = indicadores[['Ano', 'DENOM_CIA', 'margem_bruta', 'margem_liquida', 'margem_ebit']].round(decimals = 3)

CD_CONTA,Ano,DENOM_CIA,margem_bruta,margem_liquida,margem_ebit
0,2013,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,-inf,-inf,-inf
1,2013,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,44.611,11.498,26.380
2,2013,AES ELPA S.A.,3.234,1.671,2.725
3,2013,AES TIETE ENERGIA SA,18.729,11.279,17.228
4,2013,AES TIETE SA,58.289,37.714,58.289


In [118]:
indicadores_long = indicadores.melt(id_vars = ['Ano', 'DENOM_CIA'], var_name = 'indicadores')

In [119]:
indicadores_long.head()

,Ano,DENOM_CIA,indicadores,value
0,2013,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,margem_bruta,-inf
1,2013,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,margem_bruta,44.611
2,2013,AES ELPA S.A.,margem_bruta,3.234
3,2013,AES TIETE ENERGIA SA,margem_bruta,18.729
4,2013,AES TIETE SA,margem_bruta,58.289


In [121]:
# Graficos
colors = {
    'blue': '#282f6b',
    'red': '#b22200',
    'yellow': '#eace3f',
    'green': '#224f20',
    'purple':'#5f487c',
    'orange': '#b35c1e',
    'turquoise': '#419391',
    'green-two': '#839c56',
    'light-blue': '#3b89bc',
    'gray': '#666666'
    }

In [122]:
# Criando Graficos
(
    ggplot(indicadores_long, aes(x = 'Ano',
                                 y = 'value',
                                 color = 'indicadores'))
    + geom_line(size = 1)
    +  facet_wrap('DENOM_CIA', scales = 'free')
)

KeyboardInterrupt: 